In [2]:
import requests

res = requests.post('https://api-seller.ozon.ru/v2/category/tree', headers={
    'Client-Id': '589954', 'Api-Key': '2e0076d1-1ecd-4105-b1df-d87e1a244648'
})

arr = res.json()['result']
categories = []

categories_by_id = {}

def parse(arr):
    for obj in arr:
        tmp = {}
        tmp['category_id'] = obj['category_id']
        tmp['title'] = obj['title']
        if obj['children'] and len(obj['children']) != 0:
            categories_by_id[tmp['category_id']] = obj['title']
            categories.append(tmp)
            parse(obj['children'])
        else:
            categories_by_id[tmp['category_id']] = obj['title']
            categories.append(tmp)


parse(arr)

print(len(categories))
# print(categories_by_id)


10403
{17027487: 'Зоотовары', 78360021: 'Аквариумная химия, удобрения', 33589504: 'Удобрение для аквариумных растений', 17039028: 'Химия для аквариума', 17028967: 'Средство по уходу за животными', 17036157: 'Наполнитель для туалета для грызунов, птиц', 92448839: 'Песок для купания грызунов', 94737636: 'Парфюм для животных', 17030599: 'Спрей для животных', 17030600: 'Пудра для животных', 17038998: 'Бальзам для животных', 17038995: 'Средство для уборки за животными', 17038997: 'Воск для лап', 17030596: 'Блеск для животных', 17032208: 'Кондиционер для животных', 17030601: 'Мыло для животных', 94171623: 'Маска для животных косметическая', 17030602: 'Уничтожитель запаха', 88711963: 'Корректор поведения для животных', 17038989: 'Шампунь для животных', 17036156: 'Наполнитель для кошачьих туалетов', 17030603: 'Дезодорант для животных', 17032210: 'Краска для животных', 17032205: 'Влажные салфетки для животных', 17038999: 'Средство для животных', 17032206: 'Гель для животных', 17028670: 'Корм дл

In [13]:
%pip install beautifulsoup4
%pip install selenium
%pip install pyppeteer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 83.4/83.4 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 101.4/101.4 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python310\\Scripts\\pyppeteer-install.exe' -> 'c:\\Python310\\Scripts\\pyppeteer-install.exe.deleteme'


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import asyncio
from bs4 import BeautifulSoup
from pyppeteer import launch
import os
from collections import deque


async def get_page():
    browser = await launch()
    page = await browser.newPage()
    await page.setUserAgent('Mozilla/5.0 (Windows NT 5.1; rv:5.0) Gecko/20100101 Firefox/5.0')
    return page

page = await get_page()
base = 'https://www.ozon.ru'
categories_url = 'https://www.ozon.ru/category/'
used_links = {categories_url}
found_categories = set()


In [2]:
await page.goto(categories_url, {'waitUntil': 'networkidle0'})
page_content = await page.content()

def grab_categories(page_content):
    soup = BeautifulSoup(page_content)
    links = set()
    for a in soup.find_all('a', href=True):
        if re.fullmatch('\/category\/[^\/]+\\d+\/', a['href']) is not None:
            link = base + a['href']
            if link in found_categories:
                continue

            print("Found the URL:", link)
            links.add(link)
            found_categories.add(link)

    return links


queue_categories = deque(list(grab_categories(page_content)))
found_categories = set()
used_categories = set()

Found the URL: https://www.ozon.ru/category/supermarket-25000/
Found the URL: https://www.ozon.ru/category/elektronika-15500/
Found the URL: https://www.ozon.ru/category/zhenskaya-odezhda-7501/
Found the URL: https://www.ozon.ru/category/dom-i-sad-14500/
Found the URL: https://www.ozon.ru/category/detskie-tovary-7000/
Found the URL: https://www.ozon.ru/category/odezhda-obuv-i-aksessuary-7500/
Found the URL: https://www.ozon.ru/category/apteka-6000/
Found the URL: https://www.ozon.ru/category/bytovaya-tehnika-10500/
Found the URL: https://www.ozon.ru/category/knigi-16500/
Found the URL: https://www.ozon.ru/category/stroitelstvo-i-remont-9700/
Found the URL: https://www.ozon.ru/category/krasota-i-zdorove-6500/
Found the URL: https://www.ozon.ru/category/sport-i-otdyh-11000/
Found the URL: https://www.ozon.ru/category/tovary-dlya-zhivotnyh-12300/
Found the URL: https://www.ozon.ru/category/produkty-pitaniya-9200/
Found the URL: https://www.ozon.ru/category/avtotovary-8500/
Found the URL: 

In [3]:
used_categories = set(map(lambda s: s.strip(), filter(None, open(
    './used_categories.txt', 'r').readlines())))
found_categories = set(map(lambda s: s.strip(), filter(None, open(
    './found_categories.txt', 'r').readlines())))
queue_categories = deque(map(lambda s: s.strip(), filter(None, open(
    './queue_categories.txt', 'r').readlines())))


In [4]:



while len(queue_categories) != 0:
    url = queue_categories.popleft()
    if url in used_categories:
        continue
    used_categories.add(url)
    try:
        await page.goto(url, {'waitUntil': 'networkidle0'})
        page_content = await page.content()
        links = grab_categories(page_content)
        queue_categories.extend(list(links))
        open('./used_categories.txt', 'w+').write('\n'.join(used_categories), )
        open('./found_categories.txt', 'w+').write('\n'.join(found_categories))
        open('./queue_categories.txt', 'w+').write('\n'.join(queue_categories))
    except:
        print('timeout error!')
        queue_categories.extend(url)
        await browser.close()
        page = await get_page()
        




Found the URL: https://www.ozon.ru/category/planshety-samsung-galaxy-tab-2-10-1-gt-p5100/
Found the URL: https://www.ozon.ru/category/planshety-samsung-galaxy-tab-s7/
Found the URL: https://www.ozon.ru/category/planshety-samsung-galaxy-tab-a7/
Found the URL: https://www.ozon.ru/category/posudomoechnye-mashiny-10515/
Found the URL: https://www.ozon.ru/category/duhovye-shkafy-10510/
Found the URL: https://www.ozon.ru/category/plity-10509/
Found the URL: https://www.ozon.ru/category/kulery-dlya-vody-i-aksessuary-10516/
Found the URL: https://www.ozon.ru/category/aksessuary-dlya-krupnoy-bytovoy-tehniki-10794/
Found the URL: https://www.ozon.ru/category/vinnye-shkafy-10506/
Found the URL: https://www.ozon.ru/category/sigarnye-shkafy-10505/
Found the URL: https://www.ozon.ru/category/termoshkafy-dlya-hraneniya-produktov-37124/
Found the URL: https://www.ozon.ru/category/kuhonnye-vytyazhki-10512/
Found the URL: https://www.ozon.ru/category/panel-dlya-morozilnoy-kamery-holodilnika-ariston2200/

In [ ]:
await browser.close()